# **Проект исследования вакансий Junior аналитиков с сайта LinkedIn**

## **Цель исследования**

Визуализировать информацию о рынке вакансий для аналитиков в Европе, в разрезе:
- по стране и по типу занятости
- общее количество вакансий
- нанимающих компаний с указанием количества вакансий,
- ТОП 10 сфер деятельности компаний, которые нанимают аналитиков
- размер компаний и количество вакансий
- требуемые хард скилы
- Дополнительно (необязательно) - зависимость количества кандидатов на вакансию от даты публикации


### **Описание  данных:**

Предоставлен датасет с сырыми данными после парсинга сайта LinkedIn, который необходимо обработать и создать датасет, содержащий:
- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию

### **План исследования**

Работать будем в следующем порядке:
- обработка исходного файла и выделение в нем требуемых признаков
- проверка наличия дубликатов
- проверка наличия нерелевантных вакансий
- удаление ненужных атрибутов
- создание дашборда

In [31]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
from urllib.parse import urlencode 
import re
import requests
from calendar import day_name
import plotly
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline

In [87]:
# будем отображать цифры без экспоненты и только с 2 знаками после запятой
pd.options.display.float_format ='{:,.2f}'.format

## **Импорт данных**

In [2]:
# используя конструкцию try-ecxept для возможности открыть локально и с внешнего источника
try:
    df = pd.read_excel(r'C:\Users\rosto\OneDrive\Документы\Аналитика\data_for_Sigma.xlsx')
    
except:

    # используем api
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/i/HAryA9iCXncGdw'
    # получаем url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # загружаем файл в df
    download_response = requests.get(download_url)
    mark_events = pd.read_excel(download_url)

# **Общий вывод исследования**

В целом по исследованным данным можно сделать следующие выводы:
- подавляющая часть трафика на сайт это платный рекламный трафик
- трафик благодаря органическому поиску дает в 10 раз меньший трафик, чем рекламный
- рекламный трафик, как мы видели и в данных Директа неравномерный по дням, есть высокие пики, есть отсутствие заходов на сайт вообще
- самые популярные кампании те, которые касаются поиска ФН, у этих же кампаний большая глубина просмотра и среднее время на сайте
- у кампаний из РСЯ высокий % отказов и как правило короткие сессии
- визитов из РСЯ в 2 раза больше, чем из платного поиска, но эти визиты очень короткие, с высоким % отказов и низкой глубиной просмотра
- есть список платформ из РСЯ, которые необходимо исключить из показов. Показов станет меньше, но их эффективность повысится
- ¾ посетителей сайта заходят и уходят с него с одной и той же страницы, проводя на ней около 1 минуты. Больше всего времени проводят на страницах аренды, меньше всего на страницах ФН во Владимире и кассы Эвотор 7-3
- самая высокая конверсия (выше 10%)  в действие клик по номеру телефона – у страниц информации по ФН. У всех остальных страниц конверсия около 1%
